In [3]:
import numpy as np
import pandas as pd

directory = '../data'
mission_data = pd.read_csv(f'{directory}/clean-data.csv', na_values="*", index_col=0)
mission_data.iloc[-1]

millis                          1.700444e+07
year                            2.021000e+03
month                           9.000000e+00
day                             2.500000e+01
hour                            2.000000e+01
minute                          4.100000e+01
second                          3.700000e+01
voltage                         4.400000e+00
satellites                      1.200000e+01
hdop                            7.500000e-01
gps_altitude_m                  3.429000e+02
course                          1.121000e+02
speed_mps                       3.500000e+00
latitude                        4.718886e+01
longitude                      -1.194382e+02
temperature_C                   2.931000e+01
pressure_Pa                     9.685867e+04
pressure_altitude_m             3.786700e+02
humidity_RH                     2.099000e+01
mission_millis                  1.682929e+07
better_pressure_altitude_m      3.809644e+02
temp_adjusted_altitude_m        6.700543e+02
blended_al

In [4]:
burst_index = np.argmax(mission_data['gps_altitude_m'])
burst_time = mission_data.index[burst_index]

ascent_phase = mission_data[mission_data.index < burst_time]
descent_phase = mission_data[mission_data.index > burst_time]

ascent_rate = ascent_phase['ascent_rate_mps']
descent_rate = descent_phase['ascent_rate_mps'] * -1

burst_row = mission_data.iloc[burst_index]
burst_altitude = burst_row['altitude_m']

print(burst_altitude)

33669.33240871549


In [6]:
import simplekml
kml = simplekml.Kml()
kml.document.name = "Graupel-1 Actual Flight Path"

def create_point(row, name):
    point = kml.newpoint(name=name)
    latitude = row['latitude']
    longitude = row['longitude']
    altitude = row['altitude_m']
    time = row.name
    coords = row[['longitude','latitude','altitude_m']].to_numpy()
    coords.resize(1,3)
    point.coords = coords
    point.description = f'{name} at {latitude:0.4f}, {longitude:0.4f}, {altitude:0.0f}m at {time}'
    point.altitudemode = simplekml.AltitudeMode.absolute
    
def summary_stats(series):
    return f'{series.mean():0.1f}m/s ({series.max():0.1f} max)'

create_point(mission_data.iloc[0], 'Launch')
create_point(burst_row, 'Burst')
create_point(mission_data.iloc[-1], 'Landing')

every_minute_data = pd.concat([mission_data.iloc[::60], mission_data.iloc[[-1]]])

path = kml.newlinestring(name='Flight path')
path.description = f'Ascent rate: {summary_stats(ascent_rate)}, descent rate: {summary_stats(descent_rate)}'
path.coords = every_minute_data[['longitude','latitude','altitude_m']].to_numpy()
path.extrude = 1
path.altitudemode = simplekml.AltitudeMode.absolute
path.style.linestyle.color = "ffff0018"
path.style.linestyle.width = 4
path.style.polystyle.color = "ffff0018"

kml.save(f'{directory}/flight-path.kml')

In [7]:
print(kml.kml(format=True))

<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2">
    <Document id="1">
        <Style id="10">
            <LineStyle id="11">
                <color>ffff0018</color>
                <colorMode>normal</colorMode>
                <width>4</width>
            </LineStyle>
            <PolyStyle id="12">
                <color>ffff0018</color>
                <colorMode>normal</colorMode>
                <fill>1</fill>
                <outline>1</outline>
            </PolyStyle>
        </Style>
        <name>Graupel-1 Actual Flight Path</name>
        <Placemark id="3">
            <name>Launch</name>
            <description>Launch at 47.1607, -120.8484, 683m at 2021-09-25 17:28:51</description>
            <Point id="2">
                <coordinates>-120.84838104,47.1606903,682.9385557667099</coordinates>
                <altitudeMode>absolute</altitudeMode>
            </Point>
        </Placemark>
      